## Carga de data

In [2]:
import pandas as pd
import os

path_user = os.path.join('..','data','clear','users_items.csv.gz')
path_steam_games = os.path.join('..','data','clear','steam_games.csv.gz')

In [3]:
## Cargamos la data necesaria para solucionar este problema.
steam_colums = ['id','genres','release_year']
steam_games = pd.read_csv(path_steam_games, usecols = steam_colums)

user_item_cols = ['item_id','user_id','playtime_forever']
user_items = pd.read_csv(path_user, usecols = user_item_cols)


## renombraremos la columna id por steam_id
steam_games.rename(columns={'id':'item_id'}, inplace=True)


In [4]:
user_items

,user_id,playtime_forever,item_id
0,76561197970982479,0.10,10.0
1,76561197970982479,0.00,20.0
2,76561197970982479,0.12,30.0
3,76561197970982479,0.00,40.0
4,76561197970982479,0.00,50.0
...,...,...,...
5110791,76561198329548331,0.00,373330.0
5110792,76561198329548331,0.05,388490.0
5110793,76561198329548331,0.07,521570.0
5110794,76561198329548331,0.05,519140.0


In [193]:
steam_users = steam_games.merge(user_items,
                              on='item_id',
                              how='inner')

In [199]:
steam_users = steam_users[steam_users['playtime_forever'] > 0 ]

In [200]:
genres_unique =  steam_games['genres'].str.split(', ').explode().unique()

In [70]:
steam_games['genres'].str.split(', ').explode().value_counts().head(10)

Indie               16783
Action              12320
Casual               9471
Adventure            9344
Simulation           7508
Strategy             7289
RPG                  5552
Singleplayer         4268
Multiplayer          2283
Great Soundtrack     2185
Name: genres, dtype: int64

In [106]:
len(genres_unique)

339

In [163]:
genres = 'Action'

user_max = steam_users[steam_users['genres'].str.contains(genres)][['user_id','playtime_forever']].groupby('user_id').sum().nlargest(1,'playtime_forever').index

In [161]:
user_max

array(['Sp3ctre'], dtype=object)

In [73]:
play_to_years = steam_users[(steam_users['user_id'] == user_max)][['release_year','playtime_forever']].groupby('release_year').sum()

In [166]:
play_to_years.to_dict()

def user_max_time(genres):
  try:
    user_max = steam_users[steam_users['genres'].str.contains(genres)][['user_id','playtime_forever']].groupby('user_id').sum().nlargest(1,'playtime_forever').index[0]
    print(genres)
    return(user_max)
  except IndexError:
    return('Nadie')


def max_time_for_year(user):
  if user != 'Nadie':
    return(steam_users[(steam_users['user_id'] == user)][['release_year','playtime_forever']].groupby('release_year').sum().to_dict())
  

In [202]:
tabla_1 = pd.DataFrame(index=genres_unique,columns=['nombre'])

In [203]:
tabla_1 = tabla_1.reset_index()

In [204]:
tabla_1

,index,nombre
0,Strategy,NaN
1,Action,NaN
2,Indie,NaN
3,Casual,NaN
4,Simulation,NaN
...,...,...
334,Faith,NaN
335,Tutorial,NaN
336,Steam Machine,NaN
337,Bikes,NaN


In [ ]:

tabla_1['nombre'] = tabla_1['index'].apply(user_max_time)


In [213]:
tabla_1.to_csv('02_nombres__max.csv.gz')

In [220]:
tabla_1['year'] = tabla_1['nombre'].apply(max_time_for_year)

In [223]:
tabla_1[tabla_1['nombre']!='Nadie'].to_csv('02_endpoint_full.csv.gz')

In [225]:
tabla_1

,index,nombre,year
0,Strategy,shinomegami,"{'playtime_forever': {1988: 2.27, 1993: 513.71..."
1,Action,Sp3ctre,"{'playtime_forever': {1995: 3.62, 1997: 4.71, ..."
2,Indie,REBAS_AS_F-T,"{'playtime_forever': {1998: 0.02, 1999: 3.75, ..."
3,Casual,REBAS_AS_F-T,"{'playtime_forever': {1998: 0.02, 1999: 3.75, ..."
4,Simulation,shinomegami,"{'playtime_forever': {1988: 2.27, 1993: 513.71..."
...,...,...,...
334,Faith,Nadie,None
335,Tutorial,Nadie,None
336,Steam Machine,REBAS_AS_F-T,"{'playtime_forever': {1998: 0.02, 1999: 3.75, ..."
337,Bikes,Nadie,None


In [269]:
import ast
def endpoint6(genres):
  
  consulta_final = pd.read_csv('02_endpoint_full.csv.gz',index_col=['index'])
    
  user_max = consulta_final.loc['Action'].nombre
  
  diccionario = ast.literal_eval(consulta_final.loc['Action'].year)

  diccionario['Horas_Jugadas']  = diccionario.pop('playtime_forever')

  return(f"Usuario con más horas jugadas para Género {genres} : {user_max},\n Horas jugadas:{[str(diccionario)]}")

In [270]:
endpoint6('Action')

'Usuario con más horas jugadas para Género Action : Sp3ctre,\n Horas jugadas:["{\'Horas_Jugadas\': {1995: 3.62, 1997: 4.71, 1999: 10.5, 2000: 1177.4, 2001: 4.17, 2002: 5.890000000000001, 2003: 129.06, 2004: 2134.04, 2005: 355.65, 2006: 1536.3999999999999, 2007: 1879.91, 2008: 170.28, 2009: 1818.46, 2010: 1385.72, 2011: 2591.25, 2012: 6404.280000000001, 2013: 3452.12, 2014: 2285.46, 2015: 8579.77, 2016: 778.46, 2017: 722.12}}"]'